In [1]:
import datetime
from nowtrade.symbol_list import StockList
from nowtrade.data_connection import MongoDatabaseConnection
from nowtrade import dataset 
from nowtrade import technical_indicator
from nowtrade import criteria
from nowtrade import criteria_group
from nowtrade import trading_profile
from nowtrade import trading_amount
from nowtrade import trading_fee
from nowtrade import strategy
from nowtrade import ensemble
from nowtrade.action import Long
from nowtrade.action import Short
from nowtrade.action import LongExit
from nowtrade.action import ShortExit

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
from nowtrade import data_connection

In [14]:
sl=StockList(['BSE'])
symbol= sl.get('BSE')

In [15]:
train_start = datetime.datetime(2008, 01, 03)
train_end = datetime.datetime(2016,01, 01)
train_dataset= pd.read_csv('BSESN.csv')
#train_dataset = dataset.Dataset(sl, MongoDatabaseConnection(), train_start, train_end)
#train_dataset=dc.get_data(symbol,train_start,train_end)

In [16]:
#train_dataset.load_data()

In [17]:
train_dataset=train_dataset.set_index(train_dataset['Date'])

In [18]:
train_dataset=train_dataset.drop('Date',1)

In [19]:
train_dataset.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2017-05-02,30021.490234,30069.240234,29804.119141,29921.179688,11700,29921.179688
2017-04-28,30064.599609,30067.640625,29848.210938,29918.400391,11300,29918.400391
2017-04-27,30141.390625,30184.220703,29973.400391,30029.740234,10700,30029.740234
2017-04-26,30030.199219,30167.089844,29968.570312,30133.349609,10900,30133.349609
2017-04-25,29825.140625,29961.820312,29780.839844,29943.240234,8600,29943.240234


In [27]:
tt=train_dataset.sort_index(ascending=True)

In [40]:
train_data=tt.loc['2006-01-01':'2016-01-01']

In [43]:
test_data=tt.loc['2016-01-01':'2016-05-01']

In [45]:
test_data.tail()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-04-25,25891.029297,25891.029297,25585.929688,25678.929688,8200,25678.929688
2016-04-26,25604.919922,26055.000000,25549.050781,26007.300781,11200,26007.300781
2016-04-27,25956.419922,26092.929688,25885.240234,26064.119141,11200,26064.119141
2016-04-28,26078.279297,26100.539062,25561.169922,25603.099609,14600,25603.099609
2016-04-29,25612.910156,25755.429688,25424.029297,25606.619141,14600,25606.619141


In [46]:
#rf = ensemble.Ensemble(['Open', 'High', 'Low',  'Volume'], ['Close'])

In [66]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score,recall_score

In [ ]:
train_values=train_data

In [49]:
train_columns=['Open', 'High', 'Low',  'Volume']
train_test_column=['Close']

In [51]:
from sklearn.ensemble import RandomForestClassifier

In [65]:
rf=RandomForestClassifier(n_jobs=2,max_depth=7,n_estimators=500)

In [62]:
rf.fit(train_data.drop('Close',1),np.asarray(train_data['Close'],dtype="|S6"))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [64]:
rf.

<bound method RandomForestClassifier.score of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)>

In [63]:
class MongoDatabaseConnection(DataConnection):
    """
    MongoDB connection to retrieve data.
    """
    def __init__(self, host='127.0.0.1', port=27017, database='stock-data', \
                 username=None, password=None):
        DataConnection.__init__(self)
        from pymongo import MongoClient
        self.connection = None
        self.database = None
        self.host = host
        self.port = port
        self.database = database
        self.username = username
        self.password = password
        self.connection = MongoClient(host, port)
        self.database = self.connection[database]

    def get_data(self, symbol, start, end, symbol_in_column=True):
        """
        Returns a dataframe of the symbol data requested.
        """
        from pymongo import ASCENDING
        symbol = str(symbol).upper()
        results = self.database[symbol].find({'_id': \
                              {'$gte': start, '$lte': end}}\
                              ).sort('datetime', ASCENDING)
        ret = pd.DataFrame.from_dict(list(results))
        if len(ret) < 1:
            raise NoDataException()
        ret.rename(columns={'open': 'Open', \
                            'high': 'High', \
                            'low': 'Low', \
                            'close': 'Close', \
                            'volume': 'Volume', \
                            'adj_close': 'Adj Close', \
                            '_id': 'Date'}, \
                           inplace=True)
        ret = ret.set_index('Date')
        if symbol_in_column:
            ret.rename(columns=lambda name: '%s_%s' %(symbol, name), inplace=True)
        return ret

    def set_data(self, data_frame, symbols, volume=True, adj_close=True):
        """
        Stores Open, Close, High, Low, Volume, and Adj Close of
        symbols specified using the data in the DataFrame provided.
        Typically you'd pull data using another connection and
        feed it's data_frame to this function in order to store
        the data in a local MongoDB.
        """
        for symbol in symbols:
            symbol = str(symbol).upper()
            if adj_close:
                data = data_frame.loc[:, ['%s_Open' %symbol, \
                                          '%s_Close' %symbol, \
                                          '%s_High' %symbol, \
                                          '%s_Low' %symbol, \
                                          '%s_Volume' %symbol, \
                                          '%s_Adj Close' %symbol]]
                data.columns = ['open', 'close', 'high', 'low', 'volume', 'adj_close']
            elif volume:
                data = data_frame.loc[:, ['%s_Open' %symbol, \
                                          '%s_Close' %symbol, \
                                          '%s_High' %symbol, \
                                          '%s_Low' %symbol, \
                                          '%s_Volume' %symbol]]
                data.columns = ['open', 'close', 'high', 'low', 'volume']
            else:
                data = data_frame.loc[:, ['%s_Open' %symbol, \
                                          '%s_Close' %symbol, \
                                          '%s_High' %symbol, \
                                          '%s_Low' %symbol]]
                data.columns = ['open', 'close', 'high', 'low']
            for row in data.iterrows():
                values = dict(row[1])
                values['_id'] = row[0]
                self.database[symbol].insert(values)
                
def populate_mongo_day(symbols, start, end, database='symbol-data'):
    """
    Helper function to populate a local mongo db with daily stock data.
    Uses the YahooConnection class.
    """
    mgc = MongoDatabaseConnection(database=database)
    for symbol in symbols:
        symbol = symbol.upper()
        yahoo = YahooConnection()
        try:
            data = yahoo.get_data(symbol, start, end)
            mgc.set_data(data, [symbol])
        except Exception, error: # pylint: disable=broad-except
            print 'Error for %s (%s - %s): %s' %(symbol, start, end, error)
            


In [51]:
import datetime
from nowtrade import logger

In [65]:
sl=StockList(['BSE'])
symbol= sl.get('BSE')

In [35]:
populate_mongo_day(symbol,train_start,train_end)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:66: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [56]:
train_start = datetime.datetime(2008, 01, 03)
train_end = datetime.datetime(2016,01, 01)
symbol=['BSE']

In [66]:
train_dataset1 = dataset.Dataset(symbol, MongoDatabaseConnection(), train_start, train_end)

In [62]:
train_dataset1=MongoDatabaseConnection().get_data(symbol,train_start,train_end)

NoDataException: 

In [67]:
train_dataset1.load_data()

TypeError: 'Symbol' object is not iterable